<a href="https://colab.research.google.com/github/rodchimb/CallManagerTraces/blob/master/Day_5_Full_Backup_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# day5_complete_backup.py
# This is the final project, combining all lessons into a single,
# practical script to back up network device configurations.
#
# Prerequisite: Make sure you have Netmiko installed.
#
# Changes:
# - Reads device information from a JSON file.
# - Loops through multiple devices.
# - Uses a `try...except` block to handle errors for each device.
# - Saves the configuration to a file with a timestamp.

import json
import datetime
from netmiko import ConnectHandler

# Step 1: Define the device list filename.
DEVICES_FILE = 'devices.json'

def backup_device(device):
    """
    Connects to a single device and backs up its running configuration.
    """
    try:
        # Step 2: Establish the connection using ConnectHandler.
        # **device unpacks the dictionary to pass its keys as arguments.
        net_connect = ConnectHandler(**device)
        print(f"\nSuccessfully connected to {device['host']}")

        # Step 3: Run the command to get the configuration.
        print("Grabbing the running configuration...")
        output = net_connect.send_command('show run')

        # Step 4: Create a timestamped filename for the backup.
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        filename = f"{device['host']}_backup_{timestamp}.txt"

        # Step 5: Write the configuration to the new file.
        with open(filename, 'w') as f:
            f.write(output)
        print(f"Configuration saved to {filename}")

        # Step 6: Disconnect.
        net_connect.disconnect()

    except Exception as e:
        # Step 7: Handle any errors gracefully.
        print(f"Failed to connect or backup device {device['host']}. Error: {e}")

if __name__ == "__main__":
    try:
        # Step 8: Read the device information from the JSON file.
        with open(DEVICES_FILE, 'r') as f:
            devices = json.load(f)

        # Step 9: Loop through all devices in the list.
        for device_info in devices:
            device_info['device_type'] = 'cisco_ios'
            # Call the function to backup the device.
            backup_device(device_info)
    except FileNotFoundError:
        print(f"Error: The file '{DEVICES_FILE}' was not found. "
              "Please make sure it's in the same directory.")
    except json.JSONDecodeError:
        print(f"Error: The file '{DEVICES_FILE}' contains invalid JSON. "
              "Please check its format.")